In [ ]:
!pip install -q -U transformers accelerate peft bitsandbytes datasets sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="Critic.json")
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

BASE = "mistralai/Mistral-7B-Instruct-v0.2"  # or smaller model if gated
# If gated, authenticate with hf token in Colab or use a different base like 'meta-llama' or 'falcon-1b' if needed.

tokenizer = AutoTokenizer.from_pretrained(BASE, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

def preprocess(example):
    inp = example["input"]
    prompt = f"Instruction: {example['instruction']}\nInput: {inp}\nOutput:"
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

tokenized = dataset.map(preprocess, remove_columns=dataset["train"].column_names)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Map:   0%|          | 0/920 [00:00<?, ? examples/s]

Map:   0%|          | 0/103 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

model = AutoModelForCausalLM.from_pretrained(
    BASE,
    load_in_4bit=True,           # if GPU supports bitsandbytes + 4bit
    device_map="auto"
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],  # adjust for model architecture
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./critic_finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    logging_steps=50,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",
    optim="paged_adamw_32bit"
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    data_collator=data_collator
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 1032221685 (1032221685-mit-world-peace-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,2.030000
100,1.436400
150,1.259500
200,1.201900
250,1.098500
300,1.014800
350,0.993600
400,0.994500
450,0.987700


TrainOutput(global_step=460, training_loss=1.2194601991902227, metrics={'train_runtime': 3292.7877, 'train_samples_per_second': 0.559, 'train_steps_per_second': 0.14, 'total_flos': 4.021212690776064e+16, 'train_loss': 1.2194601991902227, 'epoch': 2.0})

In [ ]:
model.save_pretrained("./critic_finetuned")
tokenizer.save_pretrained("./critic_finetuned")

('./critic_finetuned/tokenizer_config.json',
 './critic_finetuned/special_tokens_map.json',
 './critic_finetuned/chat_template.jinja',
 './critic_finetuned/tokenizer.model',
 './critic_finetuned/added_tokens.json',
 './critic_finetuned/tokenizer.json')

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

base = AutoModelForCausalLM.from_pretrained(BASE, device_map="auto", offload_folder="./offload", dtype=torch.float16)
peft_model = PeftModel.from_pretrained(base, "./critic_finetuned", offload_folder="./offload")
merged = peft_model.merge_and_unload()
merged.save_pretrained("./critic_merged")
tokenizer = AutoTokenizer.from_pretrained(BASE)
tokenizer.save_pretrained("./critic_merged")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 110.12 MiB is free. Process 6525 has 14.63 GiB memory in use. Of the allocated memory 14.41 GiB is allocated by PyTorch, and 93.72 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 👇 CHANGE THIS TO YOUR COLAB MERGED MODEL FOLDER
MODEL_PATH = "/content/critic_finetuned"

print("Loading Critic model from:", MODEL_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)


def critique(question, thinker_answer):
    prompt = (
        "Instruction: Evaluate the Thinker's answer and provide critique.\n"
        f"Input: Question: {question}\n"
        f"Thinker: {thinker_answer}\n"
        "Output:"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")

    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=False,
        temperature=0.7
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# 🔽 Manual Testing
if __name__ == "__main__":
    q = input("Question: ")
    t = input("Thinker Answer: ")

    print("\nCritic Output:\n", critique(q, t))

Loading Critic model from: /content/critic_finetuned


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/critic_finetuned'. Use `repo_type` argument if needed.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_PATH = "/content/critic_finetuned"  # update path

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH).to("cpu")

def generate_critique(question, thinker_answer, max_new_tokens=140):
    prompt = f"Instruction: Evaluate the Thinker's answer and provide constructive critique.\nInput: Question: {question}\nThinker: {thinker_answer}\nOutput:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # clean: remove prompt echo if present
    return text.split("Output:")[-1].strip()

if __name__ == "__main__":
    q = input("Question: ")
    t = input("Thinker Answer: ")
    print("\nCritic:", generate_critique(q, t))